# Детектор дефектов проката стали (Для портфолио)

    Оригинальный датасет взят из соревнования https://www.kaggle.com/c/severstal-steel-defect-detection/
  
    На руках у нас датасет с множеством картинок 1280 x 204 х 3, в большинстве своем серых тонов или темных (требуется нормализация).
    
    Представены дефекты металлических поверхностей в 4 классах [1, 2, 3, 4]
    И также представлены маски дефектов для каждого класса в виде RLE
    
    Для работы я выбрал я выбрал Semantic Segmentational U-net модель (она же заняла топ-1 на соревновании), без последнего слоя активации torch.sigmoid() (добавлю на тесте, чтобы получить вероятности)
    https://arxiv.org/abs/1505.04597
    https://github.com/qubvel/segmentation_models.pytorch
    
    Это модель в энкодере которого лежит ResNet, а декодер и bottleneck'и свои.
    В блоки декодера они добавили Attention слои для создания связей между блоками

    Цель Semantic Segmentational - выделить области относящиеся к одному классу.

    Для данной задачи так же можно использовать R-CNN c Instance Segmentational, но для этого нужно будет дополнительно сегментировать каждый дефект отдельно (чуть больше кода...)

    Обучал на домашней RTX 3070, R-CNN тяжелее чем U-Net, экономим время...
    Kaggle предоставляет ГПУ, но таймаут в ~60 минут заставляет постоянно сидеть у экрана.

    Обучение одной эпохи:
    Kaggle T100 ~ 24 min
    RTX 3070    ~  7 min

    Функции потерь мы используем:
     ↓ BCEWithLogitsLoss (Binary Cross Entropy) - хорошо работает для multilabel классификации
     ↑ F_1 - также работает как метрика  для multilabel
     ↑ IoU - Intesection over Union, хорошо показывает совпадение зон маски и таргета, multilabel
     ↑ Dice - также хорошо определяет сходства двух multilabel выборокб похожа на IoU

    Обычно BCEWithLogitsLoss и Dice наиболее часто используются для моделей сегментации.

    Для дстижения лучших результатов мы делаем аугументацию изображений для каждого батча.
    
    Полученные результаты обучения модели:

    Эпоха: 14
    bce_loss:    0.02103   Характеризует расстояния между маской и таргетом  (0 к 0 он тоже приближает, если нет дефектов)
    iou_scores:  0.56879   (0,4 - считается плохим уровнем, 0,7 - хороший, 0,9 - отличный)

    f_scores:    0.46668   в 46% случаев я угадываю дефект в ключе one vs rest (это в 2 раза чаще, чем если бы я это делал наугад 25%...)

    dice_scores: 0.56879   Еще одна бинарная мера сходства, очень похожая на IoU

In [ ]:
import os
import time

from typing import Optional, Tuple

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

import pandas as pd
import numpy as np
import cv2
import pickle

from albumentations import HorizontalFlip, VerticalFlip, RandomBrightnessContrast,  ShiftScaleRotate, Normalize, Compose, GaussNoise, ElasticTransform
from albumentations.pytorch.transforms import ToTensorV2

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tqdm import tqdm

!pip install segmentation_models_pytorch
import segmentation_models_pytorch as smp # https://github.com/qubvel/segmentation_models.pytorch

# Install gdown
!conda install -y gdown

%matplotlib inline

In [ ]:
!gdown --id 1R5yFU2h-55BtEE4pdxGz8z33Pqb5EidU
!gdown --id 1mR_7Jakg8_MmfEPYFX5LeieCWrwqonHB

# Смотрим датасеты

In [ ]:
# loading dataset
DATA_DIR = '../input/severstal-steel-defect-detection'

TRAIN_IMG_DIR = DATA_DIR + '/train_images'                    # Contains training images
TEST_IMG_DIR = DATA_DIR + '/test_images'                      # Contains test images

TRAIN_CSV = DATA_DIR + '/train.csv'                       # Contains real labels for training images
TEST_CSV = DATA_DIR + '/sample_submission.csv'            # Contains dummy labels for test image
TRAINED_WEIGHTS = ''                   # Contains trained weights
TRAINED_METRICS = 'last_model.pkl'     # Contains trained metrics
SUBMISSION_FILE = 'submission.csv'     # Contains trined labels for test image
PRE_TRAINED_WEIGHTS = ''


train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

In [ ]:
train_df.head()

In [ ]:
print(f'колличество записей: {len(train_df)}')

In [ ]:
test_df.head()

# EDA

In [ ]:
def survey(df_in: pd.DataFrame):
    """
    Считаем колличество масок и классов ('maskCount', 'ImageId')
    """
    df_maskCnt = pd.DataFrame({'maskCount' : df_in.groupby('ImageId').size()})
    df_out = pd.merge(df_in, df_maskCnt, on='ImageId')
    df_out = df_out.sort_values(by=['maskCount', 'ImageId'], ascending=False)

    df_out['ClassIds'] = pd.Series(dtype=object)

    for i, row_i in df_out.iterrows():
        ClassId_list = []
        for j, row_j in df_out.loc[df_out['ImageId'] == row_i['ImageId']].iterrows():
            ClassId_list.append(row_j['ClassId'])
        df_out.at[i,'ClassIds'] = ClassId_list
    return df_out

df = survey(train_df)
df.head(15)

In [ ]:
def class_id2index(val: int):
    """ Кодируем ClassId to index in masks"""
    return int(val-1)

def index2class_id(val: int):
    """ Декодируем index to ClassId in masks"""
    return int(val+1)

def counter_func(df_in):
    """
    Считаем статистику по классам на датафрейме
    """
    length = 4
    counter = np.zeros(length, dtype=int)
    total = 0
    for i in range(length):
        try:
            index = class_id2index(df_in.index[i])
            counter[index] = df_in.iloc[i, 0]  
        except:
            continue
        
        
    total = counter.sum()
    return total, counter 

In [ ]:
def get_distribution(df_in: pd.DataFrame):
    """
    Функция выдает распределение по дефектам и классам
    :param df_in:
    :return:
    """
    mask_count_df_pivot = pd.DataFrame({'ClassCount' : df_in.groupby('ImageId').size()})
    mask_count_df_pivot = pd.DataFrame({'Num' : mask_count_df_pivot.groupby('ClassCount').size()})
    mask_count_df_pivot.sort_values('ClassCount', ascending=True, inplace=True)

    ClassId_count_df = df_in.set_index(["ImageId", "ClassId"]).groupby(level='ClassId').count()

    class_total, class_counter = counter_func(ClassId_count_df)
    mask_total, mask_counter = counter_func(mask_count_df_pivot)
    return class_total, class_counter, mask_total, mask_counter


class_total, class_counter, mask_total, mask_counter = get_distribution(df)

print("""Total cnt defected: {0},
1 class defect: {1},
2 class defect: {2},
3 class defect: {3},
4 class defect: {4}
""".format(class_total, *class_counter))

print("""Total cnt images: {0},
with one defect:    {1},
with two defects:   {2},
with three defects: {3},
with four defects:  {4}
""".format(mask_total, *mask_counter))

In [ ]:
compare_cls_distrib = [[class_total, *class_counter],
                   ]

compare_defects_distrib = [[mask_total, *mask_counter],
                   ]
labels = ['Total cnt', '1 class', '2 class', '3 class', '4 class']
x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots(figsize=(15,4))

p1 = ax.bar(x, compare_cls_distrib[0], width, label='Train dataset')

ax.set_title('Class distribution')
ax.set_ylabel('cnt')
ax.set_xticklabels(['']+labels)
ax.legend()

plt.show()

**Распределение не равномерное...**

# RLE-Mask utility functions

In [ ]:
# https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def mask2rle(img: np.array):
    """
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated (start length)  RLE
    """
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    rle = ' '.join(str(x) for x in runs)
    if rle == np.nan:
        return ''
    else:
        return rle #returns a string formated (start length)

def rle2mask(mask_rle: str, input_shape: Tuple[int, int, int]=(256,1600,1)):
    
    """    
    img
    The pixel order of the line is from top to bottom from the left vertical line.
     It must be made and handed over.
     width/height should be made [height, width, ...] to fit the rows and columns.

     example when width=4, height=3
    
    s = [1,2,3,4,5,6,7,8,9,10,11,12]
        => 1,2,3 First row on the left, second row on 4,5,6
    
    mask_rle: run-length as string formated (start length)
    shape: (height,width)!!!  of array to return
    Returns numpy array, 1 - mask, 0 - background
    """

    height, width = input_shape[:2]

    """
    RLE
    Is a repetition of (start point, length), so divide it by even/odd and start point array
     Create an array of lengths.
     s[1:]: from 1 to the end
     s[1:][::2]: Skip 2 by s[1:] to get an array of extracted values.
    """

    mask = np.zeros(width * height, dtype=np.uint8)
    if mask_rle is not np.nan:
        s = mask_rle.split()
        array = np.asarray([int(x) for x in s])
        starts = array[0::2]
        lengths = array[1::2]

        for index, start in enumerate(starts):
            begin = int(start - 1)
            end = int(begin + lengths[index])
            mask[begin : end] = 1

    """
    img
    The pixel order of the line is from top to bottom from the left vertical line.
     It must be made and handed over.
     width/height should be made [height, width, ...] to fit the rows and columns.

     ex) When width=4, height=3

    s = [1,2,3,4,5,6,7,8,9,10,11,12]
        => 1,2,3 First row on the left, second row on 4,5,6

    s.reshape(4,3) :
    [[ 1  2  3]
     [ 4  5  6]
     [ 7  8  9]
     [10 11 12]]

    s.reshape(4,3).T :
    [[ 1  4  7 10]
     [ 2  5  8 11]
     [ 3  6  9 12]]
    """

    rle_mask = mask.reshape(width, height).T
    return rle_mask

# Test RLE functions
assert mask2rle(rle2mask(df['EncodedPixels'].iloc[0]))==df['EncodedPixels'].iloc[0]
assert mask2rle(rle2mask('1 1'))=='1 1'
assert mask2rle(np.zeros((256, 1600), np.float32)) == ''

In [ ]:
def build_masks(rle_labels: pd.DataFrame, input_shape: Tuple[int, int, int]=(256, 1600, 4)):
    """
    Строит маску вида [256, 1600, 4] из rle_labels входящего датафрейма
    :param rle_labels: pd.DataFrame ImageId, ClassId, RLE
    :param input_shape: (height, width) of array to return
    :return: masks #(256, 1600, 4)
    """
    masks = np.zeros(input_shape)
    for _, val in rle_labels.iterrows():
        masks[:, :, class_id2index(val['ClassId'])] = rle2mask(val['EncodedPixels'], input_shape)
    return masks #(256, 1600, 4)

def make_mask(row_id_in: int, df_in: pd.DataFrame,  input_shape_in: Tuple[int, int, int] = (256, 1600, 4)):
    """
    По индексу записи в датафрейме, возвращает, image_id и mask [256, 1600, 4]
    :param row_id_in: int
    :param df_in: pd.DatFrame
    :param input_shape_in: Tuple[int] = (256, 1600, 4))
    :return: fname, masks
    """
    fname = df_in.iloc[row_id_in].ImageId
    rle_labels = df_in[df_in['ImageId'] == fname][['ImageId', 'ClassId', 'EncodedPixels']]
    masks = build_masks(rle_labels, input_shape=input_shape_in) #(256, 1600, 4)
    return fname, masks

In [ ]:
def show_images(df_in: pd.DataFrame, img_dir: str, trained_df_in: pd.DataFrame = None):
    """
    Вьюха, показывает несколько изображений с наибольшим колличеством классов брака,
    в случаее указания trained_df_in, сравнивает два изображения по их id
    :param df_in:
    :param img_dir:
    :param trained_df_in:
    """
    local_df = df_in
    local_trained_df = trained_df_in
    columns = 1
    if type(trained_df_in) == pd.DataFrame:
        rows = 15
    else:
        rows = 10
    
    
    fig = plt.figure(figsize=(20,100))
    
    def sorter(local_df):
        local_df = local_df.sort_values(by=['maskCount', 'ImageId'], ascending=False)
        # Паказывает изображения с наибольшим колличеством классов брака
        grp = local_df['ImageId'].drop_duplicates()[0:rows]
        return grp

    ax_idx = 1
    for filename in sorter(df_in):
        if ax_idx > rows * columns * 2:
            break

        subdf = local_df[local_df['ImageId'] == filename].reset_index()
        # Выбирает файл с масками

        fig.add_subplot(rows * 2, columns, ax_idx).set_title(filename)
        img = cv2.imread(os.path.join(img_dir, filename ))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_2 = cv2.imread(os.path.join(img_dir, filename ))
        img_2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)

        # Паказывает маски из целевого датафрейма
            
        ax_idx += 1
        fig.add_subplot(rows * 2, columns, ax_idx).\
            set_title(filename + '      Highlighted defects ClassIds: ' + str(subdf['ClassIds'][0]))

        colors = [(255, 51, 51),(255,255,51), (51,255,51), (51,51,255)]
        masks = build_masks(subdf, (256, 1600, 4)) # маски (256, 1600, 4)
        masks_len = masks.shape[2] # get 4
        
        for i in range(masks_len):
            img[masks[:, :, i] == 1] = colors[i]

        plt.imshow(img)
        ax_idx += 1


        # Показывает маски из второго датафрейма (для сравнения)
        if type(trained_df_in) == pd.DataFrame:
            subdf_trained = local_trained_df[local_trained_df['ImageId'] == filename].reset_index()
            # Вибираем файлы с масками
            fig.add_subplot(rows * 2, columns, ax_idx).\
                set_title('Trained  '+ filename + '      Highlighted defects ClassIds: ' + str(subdf['ClassIds'][0]))

            colors = [(204, 51, 51),(204,204,51), (51,204,51), (51,51,204)]
            masks = build_masks(subdf_trained, (256, 1600, 4)) # маски (256, 1600, 4)
            masks_len = masks.shape[2] # get 4

            for i in range(masks_len):
                img_2[masks[:, :, i] == 1] = colors[i]

            plt.imshow(img_2)
            ax_idx += 1
        
    print("Class 1 = Red   (bubbles/splashes)",
          "Class 2 = Yelow (folds/foliations inside the metal)",
          "Class 3 = Green (scratches received during the movement of the sheet/rolled)",
          "Class 4 = Blue  (foliations, surges, carvings, significant defects)", sep='\n')
    plt.show()
    
show_images(df, TRAIN_IMG_DIR)

**Класс 1 = Красный (Пузырьки/брызги)**

**Класс 2 = Желтый  (Складки/слоения внутри металла)**

**Класс 3 = Зеленый (Царапины полученные при движении листа\проката)**

**Класс 4 = Синий (Слоения, Наплывы, карвины, существенные дефекты)**

**2й и 3й классы очень похожи между собой, визуально сложно отличить Царапину от Складки/слоения... также распределение классов сдвинуто в пользу карапин (3 класса)**

**Есть подозрения в плохой разметке датасета...**

**Распледеление классов не равномерное, но возможно оно характерно прокату.**

**Нормализовать классы по колличеству можно, но, это приведет к значительному уменьшению датасета, что скажется на качестве обученной модели.**

**Т.к. цель стоит больше в определении мест брака, пока нет смысла в выравнивании классов.**


# Datasets & Data Loaders

In [ ]:
class SteelDataset(Dataset):
    """
    Класс по созданию датасета идущую в модель
    """
    def __init__(self,
                 df_in: pd.DataFrame,
                 data_folder_in: str,
                 mean_in: Optional[Tuple[float]],
                 std_in: Optional[Tuple[float]],
                 phase_in: str):
        """
        :param df_in: pd.DataFrame
        :param data_folder_in: str
        :param mean_in: Optional[Tuple[float]]
        :param std_in: Optional[Tuple[float]]
        :param phase_in: str
        """

        self.df = df_in
        self.root = data_folder_in
        
        self.mean = mean_in
        self.std = std_in
        self.phase = phase_in
        self.transforms = get_transforms(phase_in = self.phase, mean_in = self.mean, std_in = self.std)
        self.indices = self.df.index.tolist()

    def __getitem__(self, idx: int):
        """
        :param idx: int
        :return: img, mask
        """
        image_name, mask = make_mask(idx, self.df)
        image_path = os.path.join(self.root,  image_name)
        img = cv2.imread(image_path)

        augmented = self.transforms(image=img, mask=mask)
        img = augmented['image'] # 3x256x1600
        if self.phase == "test":
            return image_name, img
        else:    
            mask = augmented['mask'] # 256x1600x4
            mask = mask.permute(2, 0, 1)

            return img, mask

    def __len__(self):
        return len(self.indices)


def get_transforms(phase_in: str, mean_in: Optional[Tuple[float]], std_in: Optional[Tuple[float]]):
    """
    Возвращает список аугументаций для датасета
    :param phase_in: str
    :param mean_in: Optional[Tuple[float]
    :param std_in: Optional[Tuple[float]
    :return: list_trfms
    """
    list_transforms = []
    if phase_in == "train":
        list_transforms.extend(
            [
                HorizontalFlip(p=0.25), # only horizontal flip as of now
                VerticalFlip(p=0.25), 
                RandomBrightnessContrast(p=0.25),  
                ShiftScaleRotate(p=0.25), 
                GaussNoise(p=0.25), 
                ElasticTransform(p=0.25)
            ]
        )
    list_transforms.extend(
        [
            Normalize(mean=mean_in, std=std_in),
            ToTensorV2()
        ]
    )
    list_trfms = Compose(list_transforms)
    return list_trfms

def batch_mean_std(df_in: pd.DataFrame, data_folder_in: str):
    """
    На данный момент не используется, оставил на случай, вдруг решу отойти от использования стандартных

    Возвращает mean std для всего датасета, дальше применятся будет на каждом батче
    :param df_in: pd.DataFrame
    :param data_folder_in: str
    :return: images_mean, images_std
    """
    # thanks to ronaldokun from jovian.ml
    # https://jovian.ml/forum/t/assignment-4-in-class-data-science-competition/1564/268
    mean_total = []
    std_total = []
    print('df len: {0}'.format(len(df)))
    df_loc =  df_in['ImageId']
    print(df.head())
    
    for i, filename in df_loc.items():
        img = cv2.imread(os.path.join(data_folder_in, filename), cv2.COLOR_BGR2RGB)
        img = img/255.0 # to [0,1]

        mean_total.append(img.reshape(-1, 3).mean(0)) 
        std_total.append((img**2).reshape(-1, 3).mean(0))

    # те самые mean std
    images_mean =  np.array(mean_total).mean(0)
    images_std =  np.sqrt(np.array(std_total).mean(0) - images_mean**2)
    return images_mean, images_std
    

def provider(
    data_folder: str,
    df_in: pd.DataFrame,
    phase_in: str,
    mean_in: Optional[Tuple[float, float, float]] = None,
    std_in: Optional[Tuple[float, float, float]] = None,
    batch_size: int =8,
    num_workers: int =4,
):
    """
    Создает кастомный dataloader для обучения модели
    :param data_folder: str
    :param df_in: pd.DataFrame
    :param phase_in: str
    :param mean_in: Optional[Tuple[float]]
    :param std_in: Optional[Tuple[float]]
    :param batch_size: int
    :param num_workers: int
    :return: dataloader
    """

    data_folder_loc = data_folder
    df_loc = df_in
    phase_loc = phase_in
    mean_loc = mean_in
    std_loc = std_in    

    # для фаз "train" "val"
    # делаем два сета, с рандомным разбиением
    if phase_loc != "test":
        train_df_loc, val_df_loc = train_test_split(df_loc, test_size=0.2, stratify=df_in["maskCount"], random_state=42)

        df_loc = train_df_loc if phase_loc == "train" else val_df_loc

       
    image_dataset = SteelDataset(df_in = df_loc, 
                                 data_folder_in = data_folder_loc, 
                                 mean_in = mean_loc, 
                                 std_in = std_loc, 
                                 phase_in = phase_loc
                                ) # img, mask

    dataloader = DataLoader(
        image_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )

    return dataloader

def get_default_device():
    """Pick GPU if available, else CPU"""
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    return device

# Metric functions
IoU, F, Dice метрические функции, логер.

In [ ]:
class Meter:
    """
    Измеритель в модели за эпоху
    F_scores
    iou_scores
    Dice_scores
    накапливает между батчами и выдает в конце эпохи
    """
    def __init__(self, phase: str, epoch: int, ):
        self.base_threshold = 0.5 # <<<<<<<<<<< here's the threshold
        self.F_scores = []
        self.iou_scores = []
        self.Dice_scores = []

    def update(self, targets: torch.Tensor, outputs: torch.Tensor):
        """
        Считает и добавляет результаты батча в историю эпохи
        targets: torch.tensor([0..1])
        outputs: torch.tensor([-1..1])
         """
        probs = torch.sigmoid(outputs) # torch.tensor([0..1])
        
        F, iou, Dice = compute_batch(probs, targets, self.base_threshold)
        self.iou_scores.append(iou)
        self.F_scores.append(F)
        self.Dice_scores.append(Dice)

    def get_metrics(self):
        """
        Возвращает метрики накопленные в эпохе
        :return:
        """
        F = np.nanmean(self.F_scores)
        iou = np.nanmean(self.iou_scores)
        Dice = np.nanmean(self.Dice_scores)
        return F, iou, Dice

def epoch_log(phase, epoch, lr, epoch_loss, meter):
    """Логер в принт метрик за итерацию фазы"""

    F, iou, Dice  = meter.get_metrics()
    print("""Phase: {0:s}
    Epoch: {1:d} | \u2193 Lr: {2:.8} | \u2193 BCE_loss: {3:.8} | \u2191 F: {4:.4} | \u2191 IoU: {5:.4} | \u2191 Dice: {6:.4}
    """.format(phase, epoch, lr, epoch_loss, F, iou, Dice))
    return F, iou, Dice 


def confusion_matrix_and_union(prediction: torch.Tensor, truth: torch.Tensor, threshold: int):
    """
    Считает элементы confusion matrix и union для отдельно взятой моски и ее таргета
    TP, FP, FN, TN, U 
    """
    assert prediction.shape == truth.shape # [4x256x1600]
    
    #flatten label and prediction tensors
    prediction = prediction.view(-1) # [1638400]
    truth = truth.view(-1) # [1638400]
    
    prob = (prediction >= threshold).int()
    label = (truth).int()
    not_prob = (1-prob)
    not_label = (1-label)
    

    TP = (prob&label).sum().to(torch.float32)           # zero if Truth=0 or Prediction=0
    FP = (prob&not_label).sum().to(torch.float32)       # zero if Truth=0 or Prediction=1
    FN = (not_prob&label).sum().to(torch.float32)       # zero if Truth=1 or Prediction=0
    TN = (not_prob&not_label).sum().to(torch.float32)   # if Truth=1 or Prediction=1
    U = (prob|label).sum().to(torch.float32)            # zero if both are 0

    return TP, FP, FN, TN, U 

def compute_ious(TP: torch.Tensor, U: torch.Tensor):
    """Считает iou для одной ground truth mask и predicted mask"""
    
    iou = (TP + 1e-12) / (U + 1e-12)  # We smooth our devision to avoid 0/0
    
    return iou

def compute_F_score(TP: torch.Tensor, FP: torch.Tensor, FN: torch.Tensor, beta=1):
    """Считает F для одной ground truth mask и predicted mask"""
    eps = 1e-12
    precision = torch.mean(TP / (TP + FP + eps))
    recall = torch.mean(TP / (TP + FN + eps))
    
    F = ((1 + beta**2) * precision * recall / (beta**2 * precision + recall + eps))
    
    return F

def compute_Dice(prediction: torch.Tensor, truth: torch.Tensor, TP: torch.Tensor, threshold: int, reduction='mean'):
    """Считает Dice для одной ground truth mask и predicted mask"""

    assert prediction.shape == truth.shape # [4x256x1600]
    
    #flatten label and prediction tensors
    prediction = prediction.view(-1) # [1638400]
    truth = truth.view(-1) # [1638400]
    
    prob = (prediction >= threshold).float()
    label = (truth).float()
    
    intersection = TP.sum()
    prob_sum = prob.sum()
    label_sum = label.sum()
    
    Dice = (2. * intersection + 1e-12) / (prob_sum + label_sum + 1e-12)

    return Dice


def compute_batch(prediction: torch.Tensor, truth: torch.Tensor, threshold: int):
    """Считает средние F IoU Dice для батча ground truth mask и predicted mask"""
    Fs = []
    ious = []
    Dices = []

    # batch shape [4x4x256x1600]
    for preds, labels in zip(prediction, truth):
        # [4x256x1600]
        # Здесь можно увидеть персональную метриу для каждого изображения
        
        TP, FP, FN, TN, U = confusion_matrix_and_union(preds, labels, threshold)

        Fs.append(np.array(compute_F_score(TP, FP, FN, beta=1)).mean()) # [4]
        ious.append(np.array(compute_ious(TP, U)).mean()) # [4]
        Dices.append(np.array(compute_Dice(preds, labels, TP, threshold)).mean()) # [4]

    F = np.array(Fs).mean()
    iou = np.array(ious).mean()
    Dice = np.array(Dices).mean()

    return F, iou, Dice


# Model initialization

In [ ]:
model = smp.Unet("resnet50", encoder_weights="imagenet", classes=4, activation=None)
#print(type(model))

In [ ]:
# model # uncomment to take a deeper look

In [ ]:
class Trainer(object):
    """Класс для обучения и валидации модели"""
    def __init__(self, model_in, df_in: pd.DataFrame, image_root: str, epochs_in: int = 0,weights_root_in: str = TRAINED_WEIGHTS):
        """
        :param epochs_in:
        :param model_in:
        :param df_in:
        """
        self.df_local = df_in
        self.num_workers = 2
        self.batch_size = {"train": 4, "val": 4, "test": 16}
        self.train_phases = ["train", "val"]
        self.image_root = image_root
        self.epochs_in = epochs_in

        self.device = get_default_device()
        print (f'Traininig on device: {self.device}')

        self.weights = weights_root_in
        self.net = model_in

        self.dataloader = {
            phase: provider(
                data_folder=image_root,
                df_in=self.df_local,
                phase_in=phase,
                mean_in=(0.485, 0.456, 0.406),
                std_in=(0.229, 0.224, 0.225),
                batch_size=self.batch_size[phase],
                num_workers=self.num_workers
            )
            for phase in self.train_phases
        }

    def forward(self, images, targets_in=None):
        """Отправляет в модель маску и картинку, если маски нет, отправляет нули"""
        if targets_in is None:
            targets_shape = list(images.shape)  # [4,3,256,1600]
            targets_shape[1] = 4 # [4,4,256,1600]
            targets = torch.zeros(*targets_shape)
        else:
            targets = targets_in

        images = images.to(self.device)
        masks = targets.to(self.device)
        outputs = self.net(images)

        if targets_in is None:
            loss = None
        else:
            loss = self.criterion(outputs, masks)
        return loss, outputs


    def iterate(self, epoch: int, phase: str):
        """
        Итератор по бачам внутри одной эпохи
        :param epoch:
        :param phase:
        :return:
        """
        meter = Meter(phase, epoch)
        start = time.strftime("%H:%M:%S")
        print(f"Starting epoch: {epoch} | phase: {phase} | started at: {start}")
        self.net.train(phase == "train")
        dataloader = self.dataloader[phase]
        running_loss = 0.0
        total_batches = len(dataloader)
        tk0 = tqdm(dataloader, total=total_batches) # progress bar

        self.optimizer.zero_grad()
        for itr, batch in enumerate(tk0): # replace `dataloader` with `tk0` for tqdm
            images, targets = batch

            loss, outputs = self.forward(images, targets)
            loss = loss / self.accumulation_steps
            if phase == "train":
                loss.backward()
                if (itr + 1 ) % self.accumulation_steps == 0:
                    self.optimizer.step()
                    self.optimizer.zero_grad()  # обнуляем градиент для каждого батча
            running_loss += loss.item()
            outputs = outputs.detach().cpu()
            meter.update(targets, outputs)
            tk0.set_postfix(BCE_loss=(running_loss / ((itr + 1)))) #
            
        epoch_loss = (running_loss * self.accumulation_steps) / total_batches


        iou, F, Dice = epoch_log(phase, epoch, self.last_lr, epoch_loss, meter)
        if phase == 'train':
            self.lr_rates.append(self.last_lr)

        self.losses[phase].append(epoch_loss)
        self.iou_scores[phase].append(iou)
        self.F_scores[phase].append(F)
        self.Dice_scores[phase].append(Dice)
        torch.cuda.empty_cache()
        return epoch_loss

    def start(self):
        """
        Метод запуска обучения модели
        Пока без входных параметров, но в случае необходимости быстро переделывается.

        optimizer = Adam
        criterion = BCEWithLogitsLoss
        scheduler_expr = ExponentialLR  общий шедулер снижения lr
        scheduler_r_on_plateau = ReduceLROnPlateau  шедулер в случае обнаружения минимума
        early stopp = 5

        каждый батч имеет аугументацию
        сохраняет лучшую модель каждую эпоху
        Сохраняет историю обучения в конце обучения
        :return:
        """
        # параметры обучения
        self.lr = 5e-3
        self.gamma = 0.75  # lr_scheduler lr decay
        self.last_lr = float(self.lr)
        self.num_epochs = self.epochs_in # 20
        self.best_loss = float("inf")

        self.accumulation_steps = 32 // self.batch_size['train']
        self.criterion = nn.BCEWithLogitsLoss() #
        self.optimizer = optim.Adam(self.net.parameters(), lr=self.lr) #

        # постепенно снижаем lr из эпохи в эпоху
        self.scheduler_expr = optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=self.gamma) #

        # находим минимум lr в случае плато...
        self.scheduler_r_on_plateau = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, mode="min", patience=2, factor=0.1, verbose=True)

        self.best_ago_max = 5  # Early stop epochs max
        self.best_ago = 0 # Early stop epochs cnt
        self.net = self.net.to(self.device)
        cudnn.benchmark = True

        self.losses = {phase: [] for phase in self.train_phases}
        self.iou_scores = {phase: [] for phase in self.train_phases}
        self.F_scores = {phase: [] for phase in self.train_phases}
        self.Dice_scores = {phase: [] for phase in self.train_phases}
        self.lr_rates = []
        for epoch in range(self.num_epochs):
            self.iterate(epoch, "train")
            state = {
                "epoch": epoch,
                "best_loss": self.best_loss,
                "state_dict": self.net.state_dict(),
                "optimizer": self.optimizer.state_dict(),
            }
            with torch.no_grad():
                # torch.no_grad() используется вместо self.net.eval()
                val_loss = self.iterate(epoch, "val")
                self.scheduler_r_on_plateau.step(val_loss)
                self.scheduler_expr.step()
                self.last_lr = min(float(self.scheduler_r_on_plateau.optimizer.param_groups[0]['lr']), float(self.scheduler_expr.get_last_lr()[0]))

            if val_loss < self.best_loss:
                print("******** New optimal found, saving state ********")
                state["best_loss"] = self.best_loss = val_loss
                torch.save(state, f'{self.weights}{epoch}__{self.best_loss}.pth')
                self.best_ago = 0
            else:
                self.best_ago += 1

            if self.best_ago == self.best_ago_max:
                print(f'Early stopping at epoch {epoch}')
                torch.save(state, f'{self.weights}{epoch}_last__{self.best_loss}.pth')
                break

        model_results = {
            'losses' : self.losses,
            'iou_scores' : self.iou_scores,
            'f_scores' : self.F_scores,
            'dice_scores' : self.Dice_scores,
            'lr_rates' : self.lr_rates
        }

        with open(self.weights, 'wb') as fp:
            pickle.dump(model_results, fp)



In [ ]:
class Tester(Trainer):
    def __init__(self, model_in, df_in: pd.DataFrame, image_root: str, epochs_in: int = 0,
                 weights_root_in: str = TRAINED_WEIGHTS):
        """
        Класс для тестирования модели, на основе Trainer класса
        :param model_in:
        :param df_in:
        :param image_root:
        :param epochs_in:
        :param weights_root_in:
        """

        self.df_local = df_in
        self.num_workers = 2
        self.batch_size = {"train": 4, "val": 4, "test": 16}
        self.train_phases = ["train", "val"]
        self.image_root = image_root
        self.epochs_in = epochs_in

        self.device = get_default_device()
        print (f'Traininig on device: {self.device}')

        self.weigths_root = weights_root_in
        self.net = model_in

        self.dataloader = provider(
                data_folder=self.image_root,
                df_in=df_in,
                phase_in = 'test',
                mean_in=(0.485, 0.456, 0.406),
                std_in=(0.229, 0.224, 0.225),
                batch_size=self.batch_size['test'],
                num_workers=self.num_workers
            )

    def process(self, probability, threshold, min_size):
        """
        Постобработка для каждой предсказанной mask, если площадь пятна ниже  min_size, то маска будет пустой
        :param probability:
        :param threshold:
        :param min_size:
        :return:
        """
        mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1] # (256, 1600)
        num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
        predictions = np.zeros((256, 1600), np.float32)

        for c in range(1, num_component):
            p = (component == c)
            if p.sum() > min_size:
                predictions[p] = 1 # (4, 256, 1600)
        return predictions

    def rle_encode(self, fnames, batch_preds):
        for fname, pred_mask in zip(fnames, batch_preds):
            for cls, pred_mask in enumerate(pred_mask):
                pred_mask = self.process(pred_mask, self.best_threshold, self.min_size)  # (4, 256, 1600)
                rle = mask2rle(pred_mask)
                if rle != '':
                    cls = index2class_id(cls)
                    pred_loc = [fname, cls, rle]
                    self.predicted_pixels.append(pred_loc)

    def iterate(self, epoch: int, phase: str):
        """
        Итератор по единственной эпохе
        :param epoch:
        :param phase:
        :return:
        """
        start = time.strftime("%H:%M:%S")
        print(f"Starting epoch: {epoch} | phase: {phase} | started at: {start}")
        batch_size = self.batch_size[phase]
        self.net.train(phase == "test")
        dataloader = self.dataloader
        total_batches = len(dataloader)
        tk0 = tqdm(dataloader, total=total_batches) # progress bar

        for itr, batch in enumerate(tk0): # replace `dataloader` with `tk0` for tqdm
            fnames, images = batch

            _, outputs = self.forward(images)
            outputs = torch.sigmoid(outputs)
            # пропустили через последний активатор-сигмойду, в итоге приведем к [0, 1]

            outputs = outputs.detach().cpu().numpy()

            self.rle_encode(fnames, outputs)

        torch.cuda.empty_cache()


    def start(self):
        """
        Метод запуска предсказания модели
        Пока без входных параметров, но в случае необходимости быстро переделывается.

        каждый батч только нормализуется
        площадь дефектов ограничивается min_size = 100 пикселей
        преобразование масок в RLE для submission
        :return:
        """
        self.state = torch.load(self.weigths_root)
        self.net.to(self.device)  #

        self.net.load_state_dict(self.state["state_dict"])

        self.min_size = 50 # 3500 # min total of pixels on the mask

        self.best_threshold = 0.25 # 0.5
        self.predicted_pixels = [] # np.zeros((4, 256, 1600), np.float32)


        with torch.no_grad():
            self.iterate(0, "test")

        # save predictions to submission.csv
        self.df = pd.DataFrame(self.predicted_pixels, columns=['ImageId', 'ClassId', 'EncodedPixels'])
        self.df.to_csv(SUBMISSION_FILE, index=False)

In [ ]:
# model_trainer = Trainer(epochs_in = 100, model_in = model, df_in = df, image_root = TRAIN_IMG_DIR,
#                       weights_root_in = PRE_TRAINED_WEIGHTS)
# model_trainer.start()  # uncomment to start training
#
# del model_trainer
# torch.cuda.empty_cache()

# Графики обучения

In [ ]:
with open(TRAINED_METRICS, 'rb') as fp:
    metrics_score = pickle.load(fp)

def plot_lr(scores, name):
    """
    Рисует график lr
    """
    plt.figure(figsize=(15,5))
    plt.plot(range(len(scores)), scores, label=f'train {name}  log')
    plt.title(f'{name} plot'); plt.xlabel('Epoch'); plt.ylabel(f'{name} log')
    plt.xticks(range(len(scores)))
    plt.yscale("log")
    plt.legend()
    plt.show()


def plot_scores(scores, name):
    """
    Рисует графики метрик
    """
    plt.figure(figsize=(15,5))
    plt.plot(range(len(scores["train"])), scores["train"], label=f'train {name} log')
    plt.plot(range(len(scores["train"])), scores["val"], label=f'val {name} log')
    plt.title(f'{name} plot'); plt.xlabel('Epoch'); plt.ylabel(f'{name} log')
    plt.xticks(range(len(scores["train"])))
    plt.yscale("log")
    plt.legend()
    plt.show()

plot_lr(metrics_score['lr_rates'], "Learning rate")
plot_scores(metrics_score['losses'], "BCE loss")
plot_scores(metrics_score['iou_scores'], "IoU score")
plot_scores(metrics_score['f_scores'], "F score")
plot_scores(metrics_score['dice_scores'], "Dice score")

**На 14 Эпохах переобучения замечено небыло**

In [ ]:
epoch = metrics_score['losses']['val'].index(min(metrics_score['losses']['val']))

def find_best_score(metric_scores):
    """Из сохраненной истории обучения достает лучший результат"""
    epoch = metrics_score['losses']['val'].index(min(metrics_score['losses']['val']))
    bce_loss = metrics_score['losses']['val'][14]
    iou_scores = metrics_score['iou_scores']['val'][14]
    f_scores = metrics_score['f_scores']['val'][14]
    dice_scores = metrics_score['dice_scores']['val'][14]

    print("""best scores
    epoch: {0}
    bce_loss: {1}
    iou_scores: {2}
    f_scores: {3}
    dice_scores: {4}""".format(epoch, bce_loss, iou_scores, f_scores, dice_scores))

find_best_score(metrics_score)

# Test prediction and submission


**Тестируем на трейн сете обученную модель...**

In [ ]:
# Делаем датасет подобный sample_submission.csv
predicted_train_df = df.iloc[:, :1].drop_duplicates()
predicted_train_df['EncodedPixels'] =  '1 409600'
predicted_train_df['ClassId'] =  0
predicted_train_df.head()

In [ ]:
PRE_TRAINED_WEIGHTS = PRE_TRAINED_WEIGHTS+'14_0.021031175681085493.pth'

In [ ]:
model_tester = Tester(model_in = model,
                      df_in = predicted_train_df,
                      image_root = TRAIN_IMG_DIR,
                      weights_root_in = PRE_TRAINED_WEIGHTS)

In [ ]:
model_tester.start()

In [ ]:
predicted_train_df = pd.read_csv(SUBMISSION_FILE)
predicted_train_df = survey(predicted_train_df)
predicted_train_df.head()

In [ ]:
val_class_total, val_class_counter, val_mask_total, val_mask_counter = get_distribution(predicted_train_df)
print("""Total cnt defected: {0},
1 class defect: {1},
2 class defect: {2},
3 class defect: {3},
4 class defect: {4}
""".format(val_class_total, *val_class_counter))

print("""Total cnt images: {0},
with one defect:    {1},
with two defects:   {2},
with three defects: {3},
with four defects:  {4}
""".format(val_mask_total, *val_mask_counter))

In [ ]:
compare_cls_distrib = [[class_total, *class_counter],
                   [val_class_total, *val_class_counter]
                   ]

compare_defects_distrib = [[mask_total, *mask_counter],
                   [val_mask_total, *val_mask_counter]
                   ]

labels = ['Total cnt', '1 class', '2 class', '3 class', '4 class']
x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots(2, 1, figsize=(15,7))

p1 = ax[0].bar(x - width/2, compare_cls_distrib[0], 0.35, label='Ground Truth')
p2 = ax[0].bar(x + width/2, compare_cls_distrib[1], 0.35, label='Predicted')

ax[0].set_title('Class distribution over')
ax[0].set_ylabel('cnt')
ax[0].set_xticklabels(['Total cnt']+labels)
ax[0].legend()

t1 = ax[1].bar(x - width/2, compare_defects_distrib[0], 0.35, label='Ground Truth')
t2 = ax[1].bar(x + width/2, compare_defects_distrib[1], 0.35, label='Predicted')

ax[1].set_title('Class count per img distribution')
ax[1].set_ylabel('cnt')
ax[1].set_xticklabels(['']+labels)
ax[1].legend()


plt.show()


**Распределение предсказанных классов и фактических, колличество классов на изображение примерно одинаковое.**


In [ ]:
show_images(df[:], TRAIN_IMG_DIR, predicted_train_df)

**Как мы видим, предсказание часто близко к ground truth. Не идеально... Как и предпологалось 2 и 3 классы очень похожи для модели. Возможно есть какойто трюк, чтобы их лучше различать, либо объеденить их в один класс.**

**Обучали 14 эпох, но это не предел имея больше вычислительных мощностей.**

**Предсказываем на тестовом датасете**

In [ ]:
# Готовим submission датасет
model_tester = Tester(model_in = model, df_in = test_df, image_root = TEST_IMG_DIR, weights_root_in = PRE_TRAINED_WEIGHTS)
model_tester.start()

In [ ]:
predicted_df = pd.read_csv(SUBMISSION_FILE)
predicted_df = survey(predicted_df)
predicted_df.head()

In [ ]:
test_class_total, test_class_counter, test_mask_total, test_mask_counter = get_distribution(predicted_df)

print("""Total cnt defected: {0},
1 class defect: {1},
2 class defect: {2},
3 class defect: {3},
4 class defect: {4}
""".format(test_class_total, *test_class_counter))

print("""Total cnt images: {0},
with one defect:    {1},
with two defects:   {2},
with three defects: {3},
with four defects:  {4}
""".format(test_mask_total, *test_mask_counter))

In [ ]:
compare_cls_distrib = [[class_total, *class_counter],
                   [test_class_total, *test_class_counter]
                   ]

compare_defects_distrib = [[mask_total, *mask_counter],
                   [test_mask_total, *test_mask_counter]
                   ]
labels = ['Total cnt', '1 class', '2 class', '3 class', '4 class']
x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots(2, 1, figsize=(15,7))

p1 = ax[0].bar(x - width/2, compare_cls_distrib[0], 0.35, label='Train dataset')
p2 = ax[0].bar(x + width/2, compare_cls_distrib[1], 0.35, label='Test dataset')

ax[0].set_title('Class distribution')
ax[0].set_ylabel('cnt')
ax[0].set_xticklabels(['']+labels)
ax[0].set_xticklabels(['Total cnt']+labels)
ax[0].legend()

t1 = ax[1].bar(x - width/2, compare_defects_distrib[0], 0.35, label='Train dataset')
t2 = ax[1].bar(x + width/2, compare_defects_distrib[1], 0.35, label='Test dataset')

ax[1].set_title('Class count per img distribution')
ax[1].set_ylabel('cnt')
ax[1].set_xticklabels(['']+labels)
ax[1].legend()


plt.show()

In [ ]:
show_images(predicted_df, TEST_IMG_DIR)